# __Modelo Machine Learning__

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
rev1= pd.read_parquet('Reviews/rev1.gzip')
rev2= pd.read_parquet('Reviews/rev2.gzip')
rev3= pd.read_parquet('Reviews/rev3.gzip')
rev4= pd.read_parquet('Reviews/rev4.gzip')
rev5= pd.read_parquet('Reviews/rev5.gzip')
rev6= pd.read_parquet('Reviews/rev6.gzip')
rev7= pd.read_parquet('Reviews/rev7.gzip')
rev8= pd.read_parquet('Reviews/rev8.gzip')

In [ ]:
reviews= pd.concat([rev1, rev2, rev3, rev4, rev5, rev6, rev7, rev8])

In [ ]:
reviews.head()

In [ ]:
reviews.dtypes

In [ ]:
df_title = pd.read_csv('app/df_final.csv',index_col = 0, header = None, usecols = [2,5], names = ['Movie_Id', 'Name'], skiprows=[0])
df_title.head()

In [ ]:
movie_id = 'ns6'
print(df_title.loc[movie_id].Name)

In [ ]:
df_title

In [ ]:
print(len(reviews['userId'].unique()))

In [ ]:
# Calificaciones de películas por usuario

df1_by_users = reviews.groupby(['userId']).count()
df1_by_users.head()

In [ ]:
plt.hist(df1_by_users.rating, log= True)
plt.title('Calificaciones por usuario')
plt.show()

In [ ]:
reviews['rating'].hist()
plt.title('Distribución de calificaciones')
plt.show()

Veamos cuál es la película que más calificaciones tiene y cuál la que menos

In [ ]:
#Agrupar por movieId para saber el conteo de revisiones que tiene cada uno 

df1_by_movies = reviews.groupby(['movieId']).count()
df1_by_movies.head()

In [ ]:
# Pelicula con mayores calificaciones

idx_max = df1_by_movies['userId'].idxmax()
print(df_title.loc[idx_max].Name)

In [ ]:
# Pelicula con menos calificaciones
idx_min = df1_by_movies['userId'].idxmin()
print(df_title.loc[idx_min].Name)

Películas más populares (10) 

In [ ]:
df1_by_movies = reviews.groupby(['movieId']).count()
df1_by_movies.sort_values('userId', ascending = False, inplace = True)
df1_by_movies['Vistos'] = df1_by_movies['userId']
df1_by_movies.drop(columns = ['userId','rating'], inplace = True)
df1_by_movies.tail(10)

In [ ]:
df_title.shape

In [ ]:
print(df_title.loc[idx_max].Name)

In [ ]:
df1_by_movies['Titulo'] = df_title.loc[df1_by_movies.index].Name

In [ ]:
df1_by_movies.head()

In [ ]:
plt.figure(figsize = (8,4))

df1_by_movies['Vistos'].hist(log = True, bins = 20)
plt.title('Distribución de la cantidad de vistas')

plt.show()

In [ ]:
len(df1_by_movies['Vistos'])

El rango de cantidad de usuarios rankeando las películas es de 396 a 576

In [ ]:
df1_by_movies['Vistos'].max()

In [ ]:
#Se importa base como df_ para hacer revisiones y label encoder 

df_ = pd.read_csv('app/df_final.csv')

In [ ]:
df_['rating'].unique()

In [ ]:
df_.isnull().sum().value_counts

In [ ]:
import sys

In [ ]:
#def column_unique_list(data, column_name):
list_column = df_['rating'].str.split(", ").dropna().to_numpy()
list_column = np.unique(sum(list_column, []))
list_column = np.array(sorted(list_column, key=len, reverse=True))
np.set_printoptions(threshold=sys.maxsize)
#return list_column

In [ ]:
df_.loc[df_['rating'].str.contains('min'), 'rating'] = "Sin dato"

In [ ]:
df_.loc[df_['rating'].str.contains('season'), 'rating'] = "Sin dato"

In [ ]:
df_.rating.value_counts()

In [ ]:
#se crea un label encoder para la columna de rating con label encoder

from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelEncoder()
labels_rating= le.fit_transform(df_.rating)
le.classes_

In [ ]:
etiquetas= pd.DataFrame({
    "original": df_.rating,
    "codificada": labels_rating
})

In [ ]:
etiquetas.value_counts()

In [ ]:
#Sustitución de columna rating con categorias por las codificadas


df_.rating = le.fit_transform(df_.rating)



In [ ]:
df_.drop('Unnamed: 0.1', inplace=True, axis=1)

In [ ]:
df_.head()

In [ ]:
df = df_.rename(columns={'id':'movieId'})

In [ ]:
df.head()

In [ ]:
reviews.rename(columns={'rating':'score_usuario'}, inplace=True)

In [ ]:
reviews.head()

In [ ]:
extract= df[['movieId','title', 'rating']]

In [ ]:
base= reviews.merge(extract, on='movieId')

In [ ]:
base.sample(10)

In [ ]:
users = list(base['userId'].unique())
items = list(base['rating'].unique())

In [ ]:
df_g=base.groupby(by=['userId','rating']).mean()

In [ ]:
df_g.head()

In [ ]:
df_g.drop('timestamp', inplace=True, axis=1)

In [ ]:
df_g

In [ ]:
#agrupación por rating type

df1_by_rating = base.groupby(['rating','userId']).count()
df1_by_rating.sort_values('userId', ascending = False, inplace = True)
df1_by_rating['rating_vistos'] = df1_by_rating['userId']
#df1_by_movies.drop(columns = ['userId','rating'], inplace = True)
df1_by_rating.rating_vistos.tail(10)
df1_by_rating.head()

In [ ]:
df1_by_rating = base.groupby(['userId']).mean()


In [ ]:
df1_by_rating.head()

In [295]:
base.head()

,userId,score_usuario,timestamp,movieId,title,rating
0,1,1.0,1425941529,as680,the english civil war,0
1,583,4.5,1112982383,as680,the english civil war,0
2,765,5.0,889677014,as680,the english civil war,0
3,2116,3.0,836264839,as680,the english civil war,0
4,2143,3.0,1435067487,as680,the english civil war,0


In [297]:
condicion = [(base['score_usuario'] <3.0),
            (base['score_usuario']>3.0)]
valores=['Alta Preferencia','Baja preferencia']

In [298]:
base['preferencia']= np.select(condicion, valores)

# __Machine Learning__

In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from surprise import SVD
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [ ]:
from surprise import SVD
from surprise import Dataset
from sklearn.model_selection import cross_validate

In [ ]:
import sys
#!conda activate henry-bootcamp
#!conda install -c conda-forge scikit-surprise 
from surprise import Dataset
from surprise import Reader
from sklearn.model_selection import train_test_split

In [ ]:
base['score_usuario'].dtype()

__k-vecinos__

In [299]:
from sklearn.neighbors import KNeighborsClassifier

In [300]:
clf = KNeighborsClassifier(n_neighbors=50)

In [301]:
X = base[['rating', 'userId']]  # Denotamos X con mayúscula ya que 
                                                     # incluye más de un atributo
y = base.preferencia # Etiqueta a predecir

In [302]:
clf.fit(X.values,y.values)

KNeighborsClassifier(n_neighbors=50)

In [303]:
y_pred = clf.predict(X.values)

In [ ]:
print(accuracy_score(y,y_pred))

In [290]:
import numpy as np


__SVD__


In [278]:
data = base

In [279]:
trainset, testset = train_test_split(data, test_size=.25)

In [283]:
model = SVD()

In [289]:
model.fit(trainset)

AttributeError: 'SVD' object has no attribute 'fit'

In [282]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from sklearn.model_selection import train_test_split

In [ ]:
users = list(base['userId'].unique())
items = list(base['rating'].unique())

In [ ]:
reader = Reader(rating_scale= (1,5))
data_sp = Dataset(df_g, reader=reader)

In [ ]:
from surprise import SVD
model = SVD()

In [ ]:
reader = Reader()

In [ ]:
#N_filas = 100000

data = reviews

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
# Usaremos un modelo de ~~~
from surprise import SVD
model = SVD()

1.- Cálculo de simililaridad entre usuarios

Correlación de Pearson, coseno, correlación con restricciones, coeficiente de Spearman, diferencia cuadrática media (MSD) e índice  de Jaccard (JMSD)

2.- Calcular los K-Vecinos 

Usando métrica de similaridad seleccionada y se obtienen usuarios más similares al usuario activo 

3.- Calcular predicciones de los items 

A partir de los vecinos se determinan las posibles valoraciones que el usuario activo haría sobre los items que no ha votado

4.- Realizar recomendaciones 

Se eligen los items más adecuados para ser recomendados por el usuario. Predicciones más altas, más novedosas, más votadas 

In [ ]:
model.fit(trainset)

In [ ]:
predictions = model.test(testset)